In [72]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# check xgboost version
from xgboost import XGBRegressor



In [73]:
# Carregar os dados (substitua pelo seu dataset)
data = pd.read_csv('dados_correios.csv')

In [75]:
data.head()

,tipo_demanda,horario,local,volume_atendimentos,nota_avaliacao_ai,tempo_medio_atendimento,historico_demandas,estado,regiao,cidade,numero_funcionarios,mes,dia_semana,ano
0,Cartas,22,Agência Central,457,1.35,10.51,180,SP,Sudeste,Salvador,27,12,Quinta,2013
1,Outro,3,Agência Bairro,102,4.13,19.46,221,MG,Sudeste,Salvador,39,2,Terça,2012
2,Expressa,17,Agência Bairro,113,1.62,11.03,669,MG,Nordeste,Belo Horizonte,37,4,Segunda,2010
3,Expressa,4,Centro de Distribuição,363,1.69,7.19,911,SP,Sudeste,São Paulo,39,7,Quinta,2010
4,Pacotes,15,Agência Central,423,4.32,18.27,732,RJ,Nordeste,São Paulo,26,2,Quinta,2023


In [76]:
data = data.drop('cidade', axis=1)

In [77]:
data = data.drop('regiao', axis=1)

In [78]:
data = data.drop('historico_demandas', axis=1)

## Engenharia de variaveis para modelo de previsão de demanda

### Média Diária, Mensal

In [79]:
import pandas as pd

# Exemplo de DataFrame
# train = pd.read_csv('seu_arquivo.csv')

# Calcular média diária e mensal
data['diaria_avg'] = data.groupby(['tipo_demanda', 'local', 'dia_semana'])['volume_atendimentos'].transform('mean')
data['mensal_avg'] = data.groupby(['tipo_demanda', 'local', 'mes'])['volume_atendimentos'].transform('mean')


# Calcular média móvel de 7 dias
data['media_movel_7dias'] = data.groupby(['tipo_demanda', 'local'])['volume_atendimentos'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# Média por local e tipo de demanda
data['media_atendimentos_por_local_demanda'] = data.groupby(['local', 'tipo_demanda'])['volume_atendimentos'].transform('mean')
data['media_atendimentos_por_local'] = data.groupby(['local'])['volume_atendimentos'].transform('mean')
data['media_atendimentos_por_demanda'] = data.groupby(['tipo_demanda'])['volume_atendimentos'].transform('mean')



## médias móveis

In [80]:
# Sales Rolling mean sequence per item

#rolling_10 = data.groupby(['tipo_demanda'])['volume_atendimentos'].rolling(10).mean().reset_index()
#data['rolling_mean'] = rolling_10['volume_atendimentos']

# 90 last days of training rolling mean sequence added to test data
#rolling_last90 = data.groupby(['tipo_demanda','local'])['rolling_mean'].tail(90).copy()
#data['rolling_mean'] = rolling_last90.reset_index().drop('index', axis=1)

# Shifting rolling mean 3 months
#data['rolling_mean'] = data.groupby(['tipo_demanda'])['rolling_mean'].shift(90) # Create a feature with rolling mean of day - 90

In [81]:
# Excluir NaNs resultantes de operações
data = data.dropna()

In [82]:
data.head()

,tipo_demanda,horario,local,volume_atendimentos,nota_avaliacao_ai,tempo_medio_atendimento,estado,numero_funcionarios,mes,dia_semana,ano,diaria_avg,mensal_avg,media_movel_7dias,media_atendimentos_por_local_demanda,media_atendimentos_por_local,media_atendimentos_por_demanda
0,Cartas,22,Agência Central,457,1.35,10.51,SP,27,12,Quinta,2013,279.185393,284.956522,457.0,278.511002,277.263424,276.229340
1,Outro,3,Agência Bairro,102,4.13,19.46,MG,39,2,Terça,2012,287.729282,245.117647,102.0,271.370370,271.635847,272.881061
2,Expressa,17,Agência Bairro,113,1.62,11.03,MG,37,4,Segunda,2010,262.335366,250.986111,113.0,264.487148,271.635847,272.671844
3,Expressa,4,Centro de Distribuição,363,1.69,7.19,SP,39,7,Quinta,2010,288.741259,257.218391,363.0,275.578947,275.147194,272.671844
4,Pacotes,15,Agência Central,423,4.32,18.27,RJ,26,2,Quinta,2023,268.988095,267.826667,423.0,278.902036,277.263424,276.683157


## Modelo

In [83]:
# Separar as features (X) e o target (y)

X = data.drop('volume_atendimentos', axis=1)  # Substitua 'target' pelo nome da sua variável target
y = data['volume_atendimentos']

# Convertendo variáveis categóricas em dummy variables (One-Hot Encoding)
X = pd.get_dummies(X)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar um modelo de regressão (Exemplo: RandomForestRegressor)
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [84]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que você já tenha um modelo treinado chamado 'model' e um conjunto de teste 'X_test'

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Valores reais do conjunto de teste
y_test = np.array(y_test)  # Certifique-se de que y_test está em formato numpy array

# Calcular MAE
mae = mean_absolute_error(y_test, y_pred)

# Calcular MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE
rmse = np.sqrt(mse)

# Calcular R-squared
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE): 51.535135
Mean Squared Error (MSE): 4723.819701199999
Root Mean Squared Error (RMSE): 68.73004947764842
R-squared (R²): 0.7247716990492988


In [85]:
data.head()

,tipo_demanda,horario,local,volume_atendimentos,nota_avaliacao_ai,tempo_medio_atendimento,estado,numero_funcionarios,mes,dia_semana,ano,diaria_avg,mensal_avg,media_movel_7dias,media_atendimentos_por_local_demanda,media_atendimentos_por_local,media_atendimentos_por_demanda
0,Cartas,22,Agência Central,457,1.35,10.51,SP,27,12,Quinta,2013,279.185393,284.956522,457.0,278.511002,277.263424,276.229340
1,Outro,3,Agência Bairro,102,4.13,19.46,MG,39,2,Terça,2012,287.729282,245.117647,102.0,271.370370,271.635847,272.881061
2,Expressa,17,Agência Bairro,113,1.62,11.03,MG,37,4,Segunda,2010,262.335366,250.986111,113.0,264.487148,271.635847,272.671844
3,Expressa,4,Centro de Distribuição,363,1.69,7.19,SP,39,7,Quinta,2010,288.741259,257.218391,363.0,275.578947,275.147194,272.671844
4,Pacotes,15,Agência Central,423,4.32,18.27,RJ,26,2,Quinta,2023,268.988095,267.826667,423.0,278.902036,277.263424,276.683157


## Salvar o modelo

In [86]:
import pickle
pickle_out = open("model_teste.pkl", "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [87]:
import pickle
import gzip

# Supondo que `model` seja o objeto que você deseja salvar
with gzip.open('model_teste.pkl.gz', 'wb') as f:
    pickle.dump(model, f)


In [88]:
import sklearn

In [89]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.3.2.


In [90]:
data.head()

,tipo_demanda,horario,local,volume_atendimentos,nota_avaliacao_ai,tempo_medio_atendimento,estado,numero_funcionarios,mes,dia_semana,ano,diaria_avg,mensal_avg,media_movel_7dias,media_atendimentos_por_local_demanda,media_atendimentos_por_local,media_atendimentos_por_demanda
0,Cartas,22,Agência Central,457,1.35,10.51,SP,27,12,Quinta,2013,279.185393,284.956522,457.0,278.511002,277.263424,276.229340
1,Outro,3,Agência Bairro,102,4.13,19.46,MG,39,2,Terça,2012,287.729282,245.117647,102.0,271.370370,271.635847,272.881061
2,Expressa,17,Agência Bairro,113,1.62,11.03,MG,37,4,Segunda,2010,262.335366,250.986111,113.0,264.487148,271.635847,272.671844
3,Expressa,4,Centro de Distribuição,363,1.69,7.19,SP,39,7,Quinta,2010,288.741259,257.218391,363.0,275.578947,275.147194,272.671844
4,Pacotes,15,Agência Central,423,4.32,18.27,RJ,26,2,Quinta,2023,268.988095,267.826667,423.0,278.902036,277.263424,276.683157


In [91]:
grouped_data = data.groupby('dia_semana')['volume_atendimentos'].sum().sort_values(ascending = False)

In [92]:
grouped_data = data.groupby('mes')['volume_atendimentos'].sum().sort_values(ascending = False)

In [93]:
grouped_data

,volume_atendimentos
mes,
11,484716
12,482182
4,477716
5,476158
1,454754
3,453436
10,452606
6,449350
2,447898


In [ ]:
data.to_csv('Dados_tratados.csv', index = False)